In [1]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model

In [13]:
student_obj = Student.objects.get(enrollment=220283107033)
# get the semester form current term
term_obj = Term.objects.filter(status=True).first()
students_semester = Semester.objects.get(stream__branch__term=term_obj,students=student_obj)
students_semester

<Semester: Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>

In [6]:
student_obj = Student.objects.get(enrollment=220283107033)
student_obj.profile.set_password('Manav@1011')
student_obj.profile.save()

In [13]:
subjects = filter(lambda subject:subject.subject_map.is_elective,subjects)
list(subjects)

[<Subject: Mobile Computing and Wireless Communication | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Cloud Computing | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Information security | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Big Data Analytics | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Machine Learning | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Digital forensics | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>,
 <Subject: Mobile Application Development | Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>]